In [401]:
belasting_percentage_laag = 0.371
belasting_percentage_hoog = 0.495
belasting_top_bedrag = 68508

jaar = 12
vakantiegeld_13e_maand = 13.96
looptijd_hypotheek = 360 # in maanden
hypotheek_bedrag = 145000
rente_percentage = 0.015

gemiddelde_loonstijging_percentage = 1.03

algemene_heffingskorting_ondergrens = 21043
algemene_heffingskorting_middengrens = 68507
algemene_heffingskorting_middengrens_percentage = 0.05977
algemene_heffingskorting_ondergrens_bedrag = 2837

woz_waarde = 17400
woz_waardes = [127000, 132000, 133000, 132000,135000, 153000, 174000]
woz_groei = 7.8

maandsalaris_bruto = 3200
totaal_jaarsalaris = maandsalaris_bruto * vakantiegeld_13e_maand
totaal_jaarsalaris_exclusief = (maandsalaris_bruto - pensioen_heffing) * jaar
pensioen_heffing = 138
pensioen_heffing_jaar = pensioen_heffing * jaar

import pandas as pd


In [402]:
def bereken_heffingskorting(totaal_inkomen):
    if totaal_inkomen <= algemene_heffingskorting_ondergrens:
        return algemene_heffingskorting_ondergrens_bedrag
    elif totaal_inkomen <= algemene_heffingskorting_middengrens:
        return algemene_heffingskorting_ondergrens_bedrag - (algemene_heffingskorting_middengrens_percentage * (totaal_inkomen - algemene_heffingskorting_ondergrens))
    else:
        return 0

In [403]:
def bereken_arbeidskorting(arbeids_inkomen):
    arbeidskorting_beginbedrag = 4205
    arbeidskorting_percentage = 0.06
    arbeidskorting_grens = 35652
    return arbeidskorting_beginbedrag - arbeidskorting_percentage * (arbeids_inkomen - arbeidskorting_grens)


In [404]:
def bereken_belasting(totaal_inkomen):
    belasting_bedrag = 0
    if totaal_inkomen >= belasting_top_bedrag:
        schijf_1_belasting = belasting_top_bedrag * belasting_percentage_laag
        schijf_2_belasting = (totaal_inkomen - belasting_top_bedrag) * belasting_percentage_hoog
        belasting_bedrag = schijf_1_belasting + schijf_2_belasting
    else:
        belasting_bedrag = totaal_inkomen * belasting_percentage_laag
    return belasting_bedrag

In [405]:
def bereken_netto_salaris(salaris, aftrekposten, jaar):
    belastbaar_inkomen = salaris - aftrekposten
    arbeidskorting_bedrag = bereken_arbeidskorting(salaris)
    heffingskorting_bedrag = bereken_heffingskorting(salaris)
    forfait = bereken_woz_waarde(woz_waarde, jaar) * 0.05
    belastbaar_inkomen = belastbaar_inkomen + forfait
    belasting_bedrag_totaal = bereken_belasting(belastbaar_inkomen)
    belasting_bedrag_met_kortingen = belasting_bedrag_totaal - arbeidskorting_bedrag - heffingskorting_bedrag
    return salaris - belasting_bedrag_met_kortingen

In [406]:
def bereken_woz_waarde(woz_waarde, jaar):
    return woz_waarde + (jaar * woz_groei)

In [407]:
def bereken_annuiteit(totaal_bedrag, rente_percentage):
    maandrente = rente_percentage / 12
    return (maandrente / (1 - ((1 + maandrente) ** -looptijd_hypotheek))) * totaal_bedrag

In [440]:
# annuit hypotheek

def bereken_annuit_hypotheek(totaal_bedrag, rente_percentage, rentevaste_periode=30, rente_percentage_variabel = 0):
    rentevaste_maanden = rentevaste_periode * 12
    annuiteit = bereken_annuiteit(totaal_bedrag, rente_percentage)
    rente_jaren_dict = {}
    jaren = 0
    rentebedrag_jaar = 0
    for i in range(360):
        x = i + 1
        if i < rentevaste_maanden:
            rente = (totaal_bedrag * rente_percentage) / 12
        else:
            rente = (totaal_bedrag * rente_percentage_variabel) / 12
        rentebedrag_jaar += rente
        aflossing = annuiteit - rente
        totaal_bedrag = totaal_bedrag - aflossing
        if x % 12 == 0:
            jaren += 1
            rente_jaren_dict[jaren] = rentebedrag_jaar
            rentebedrag_jaar = 0

        # print('maand', x, 'totaalbedrag', "{:.0f}".format(totaal_bedrag), 'aflossing', "{:.0f}".format(aflossing), 'rente', "{:.0f}".format(rente), 'maandbedrag', "{:.0f}".format(aflossing+rente))
    return rente_jaren_dict





In [426]:
def bereken_aflosvrij_hypotheek(totaal_bedrag, rente_percentage):
    return totaal_bedrag * rente_percentage
        

In [427]:
bereken_aflosvrij_hypotheek(145000, 0.018)

2610.0

In [454]:
# stel 30 jaar rente vast met 
rente_bedragen_annuiteit_30_vast = bereken_annuit_hypotheek(hypotheek_bedrag, rente_percentage)
totaal_bedrag_aan_rente_30_vast = sum(rente_bedragen_annuiteit_30_vast.values())
totaal_bedrag_aan_rente_30_vast 

35152.749858880496

In [461]:
rente_bedragen_annuiteit_20_vast = bereken_annuit_hypotheek(hypotheek_bedrag, 0.015, 20, 0.05)
totaal_bedrag_aan_rente_20_vast = sum(rente_bedragen_annuiteit_20_vast.values())
totaal_bedrag_aan_rente_20_vast

49236.54795861744

In [462]:
rente_bedragen_annuiteit_15_vast = bereken_annuit_hypotheek(hypotheek_bedrag, 0.014, 15, 0.05)
totaal_bedrag_aan_rente_15_vast = sum(rente_bedragen_annuiteit_15_vast.values())
totaal_bedrag_aan_rente_15_vast

70018.50117084495

In [463]:
rente_bedragen_annuiteit_10_vast = bereken_annuit_hypotheek(hypotheek_bedrag, 0.0113, 10, 0.05)
totaal_bedrag_aan_rente_10_vast = sum(rente_bedragen_annuiteit_10_vast.values())
totaal_bedrag_aan_rente_10_vast

107475.78492156796

In [411]:
# for key, value in rente_bedragen_annuiteit.items():
    # print(bereken_netto_salaris(totaal_jaarsalaris, value, key), value)

In [412]:
# verschillende rentepercentages berekenen
rente_percentages_banken = {
    'ING' : 0.0096,
    'ING2' : 0.0113,
    'ING3' : 0.015,
    
}
for key, value in rente_percentages_banken.items():
    print(key, bereken_annuiteit(146000, value))

ING 466.9159725984316
ING2 478.3628096934269
ING3 503.8755072681438


In [623]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

def bereken_netto_salaris_en_belasting(hypotheek_bedrag, rente_percentage, hoeveel_jaar, type_hypotheek, rentevaste_periode = 30, rente_na_vaste = 0):
    bruto_salaris = totaal_jaarsalaris
    kolommen = ['Hypotheektype', 'Hypotheekbedrag', 'Rentepercentage', 'Rentevaste periode', 'Verstrekker', 'Jaar', 'Datum', 'Bruto', 'Netto', 'Bedrag kwijt', 'Percentage kwijt', 'Jaarlasten hypotheek', 'Netto salaris na hypotheeklasten']
    dataframe = pd.DataFrame(columns=kolommen)
    verstrekker = 'Hypothetisch'
    if (rente_percentage == 0.015 and rentevaste_periode == 20) or (rente_percentage == 0.0113 and rentevaste_periode == 10):
        verstrekker = "ING"
    for i in range(hoeveel_jaar):
        jaar = i + 1
        jaar_lasten = 0
        nu = datetime.now()
        nieuwe_datum = (nu + relativedelta(years=jaar)).strftime("%Y-%m-%d")
        if type_hypotheek == 'aflosvrij':
            netto_salaris = bereken_netto_salaris(bruto_salaris, 0, jaar)
            jaar_lasten = bereken_aflosvrij_hypotheek(hypotheek_bedrag, rente_percentage)
        elif type_hypotheek == 'annuit':
            rente_bedragen_annuiteit = bereken_annuit_hypotheek(hypotheek_bedrag, rente_percentage, rentevaste_periode, rente_na_vaste)
            netto_salaris = bereken_netto_salaris(bruto_salaris, rente_bedragen_annuiteit[jaar], jaar)
            jaar_lasten = bereken_annuiteit(hypotheek_bedrag, rente_percentage) * 12
        
        belasting_bedrag = netto_salaris - bruto_salaris
        belasting_percentage = 100 - ((netto_salaris / bruto_salaris) * 100)
        
        rij = {'Hypotheektype': type_hypotheek,
        'Hypotheekbedrag' : hypotheek_bedrag,
         'Jaar': jaar, 
         'Datum': nieuwe_datum,
         'Rentepercentage' : "{:.2%}".format(rente_percentage),
         'Rentevaste periode' : rentevaste_periode,
         'Verstrekker' : verstrekker,
         'Bruto': float("{:.2f}".format(bruto_salaris)), 
         'Netto': float("{:.2f}".format(netto_salaris)),
         'Bedrag kwijt' : float("{:.2f}".format(belasting_bedrag)),
         'Percentage kwijt' : float("{:.2f}".format(belasting_percentage)),
         'Jaarlasten hypotheek' : float("{:.2f}".format(jaar_lasten)),
         'Netto salaris na hypotheeklasten' : float("{:.2f}".format(netto_salaris - jaar_lasten)) 
         }
        dataframe = dataframe.append(rij, ignore_index = True)
        bruto_salaris = bruto_salaris * gemiddelde_loonstijging_percentage
    return dataframe
        
        

# print('Jaar', jaar, 'Bruto',"{:.0f}".format(bruto_salaris), 'Netto', "{:.0f}".format(netto_salaris), 'Bedrag kwijt',"{:.0f}".format(belasting_bedrag), 'Percentage kwijt', "{:.2f}".format(belasting_percentage), 'Jaarlasten hypotheek', "{:.0f}".format(jaar_lasten), "Netto salaris na hypotheeklasten", "{:.0f}".format(netto_salaris - jaar_lasten))
#nog berekenen met de maandlasten wat ik dan precies kwijt ben


In [624]:
bereken_netto_salaris_en_belasting(145000, 0.018, 7, 'aflosvrij')

,Hypotheektype,Hypotheekbedrag,Rentepercentage,Rentevaste periode,Verstrekker,Jaar,Datum,Bruto,Netto,Bedrag kwijt,Percentage kwijt,Jaarlasten hypotheek,Netto salaris na hypotheeklasten
0,aflosvrij,145000,1.80%,30,Hypothetisch,1,2022-04-11,44672.00,32864.27,-11807.73,26.43,2610.0,30254.27
1,aflosvrij,145000,1.80%,30,Hypothetisch,2,2023-04-11,46012.16,33546.57,-12465.59,27.09,2610.0,30936.57
2,aflosvrij,145000,1.80%,30,Hypothetisch,3,2024-04-11,47392.52,34249.35,-13143.17,27.73,2610.0,31639.35
3,aflosvrij,145000,1.80%,30,Hypothetisch,4,2025-04-11,48814.30,34973.22,-13841.08,28.35,2610.0,32363.22
4,aflosvrij,145000,1.80%,30,Hypothetisch,5,2026-04-11,50278.73,35718.80,-14559.93,28.96,2610.0,33108.80
5,aflosvrij,145000,1.80%,30,Hypothetisch,6,2027-04-11,51787.09,36486.76,-15300.33,29.54,2610.0,33876.76
6,aflosvrij,145000,1.80%,30,Hypothetisch,7,2028-04-11,53340.70,37277.76,-16062.94,30.11,2610.0,34667.76


In [625]:
df_10_annuit =  bereken_netto_salaris_en_belasting(145000, 0.0113, 30, 'annuit', 10, 0.05)

In [626]:
df_20_annuit = bereken_netto_salaris_en_belasting(145000, 0.015, 30, 'annuit', 20, 0.05)

In [627]:
df_30_annuit = bereken_netto_salaris_en_belasting(145000, 0.02, 30, 'annuit')

In [628]:
alle_dfs2 = pd.concat([df_10_annuit, df_20_annuit, df_30_annuit])

In [629]:
alle_dfs2.to_csv('hypotheek.csv', sep=';')

In [630]:
alle_dfs = pd.DataFrame()
rente_percentages = [x * 0.00075 for x in range(18, 30)]
for x in rente_percentages:
    alle_dfs =alle_dfs.append(bereken_netto_salaris_en_belasting(145000, x, 30, 'annuit', 20, 0.05))

rente_percentages_10 = [y * 0.00075 for y in range(13, 20)]
for y in rente_percentages_10:
    alle_dfs = alle_dfs.append(bereken_netto_salaris_en_belasting(145000, y, 30, 'annuit', 10, 0.05))

In [631]:
alle_dfs.to_csv('hypotheek.csv', sep=';')

In [632]:
bereken_netto_salaris_en_belasting(145000, 0.0113, 7, 'annuit')

,Hypotheektype,Hypotheekbedrag,Rentepercentage,Rentevaste periode,Verstrekker,Jaar,Datum,Bruto,Netto,Bedrag kwijt,Percentage kwijt,Jaarlasten hypotheek,Netto salaris na hypotheeklasten
0,annuit,145000,1.13%,30,Hypothetisch,1,2022-04-11,44672.00,33464.32,-11207.68,25.09,5701.04,27763.28
1,annuit,145000,1.13%,30,Hypothetisch,2,2023-04-11,46012.16,34129.42,-11882.74,25.83,5701.04,28428.38
2,annuit,145000,1.13%,30,Hypothetisch,3,2024-04-11,47392.52,34814.79,-12577.73,26.54,5701.04,29113.76
3,annuit,145000,1.13%,30,Hypothetisch,4,2025-04-11,48814.30,35521.06,-13293.25,27.23,5701.04,29820.02
4,annuit,145000,1.13%,30,Hypothetisch,5,2026-04-11,50278.73,36248.84,-14029.89,27.90,5701.04,30547.80
5,annuit,145000,1.13%,30,Hypothetisch,6,2027-04-11,51787.09,36998.79,-14788.30,28.56,5701.04,31297.76
6,annuit,145000,1.13%,30,Hypothetisch,7,2028-04-11,53340.70,37771.59,-15569.12,29.19,5701.04,32070.55


In [633]:
rente_percentages = [x * 0.00075 for x in range(13, 30)]
for x in rente_percentages:
    rente_bedragen_annuiteit_30_vast = bereken_annuit_hypotheek(hypotheek_bedrag, x)
    totaal_bedrag_aan_rente_30_vast = sum(rente_bedragen_annuiteit_30_vast.values())
    print('Rentepercentage', "{:.2f}".format(x * 100), "Totaal rente", "{:.0f}".format(totaal_bedrag_aan_rente_30_vast))

Rentepercentage 0.97 Totaal rente 22297
Rentepercentage 1.05 Totaal rente 24097
Rentepercentage 1.12 Totaal rente 25910
Rentepercentage 1.20 Totaal rente 27734
Rentepercentage 1.28 Totaal rente 29571
Rentepercentage 1.35 Totaal rente 31420
Rentepercentage 1.43 Totaal rente 33280
Rentepercentage 1.50 Totaal rente 35153
Rentepercentage 1.57 Totaal rente 37037
Rentepercentage 1.65 Totaal rente 38934
Rentepercentage 1.73 Totaal rente 40842
Rentepercentage 1.80 Totaal rente 42763
Rentepercentage 1.88 Totaal rente 44695
Rentepercentage 1.95 Totaal rente 46639
Rentepercentage 2.02 Totaal rente 48595
Rentepercentage 2.10 Totaal rente 50562
Rentepercentage 2.18 Totaal rente 52541
